Building the SmartTag models
===


## Initial checks

In [ ]:
source .venv/bin/activate

In [ ]:
python setup.py install

In [ ]:
ls

In [ ]:
ls models

In [ ]:
ls data

In [ ]:
ls data4th

In [ ]:
smtag-neo2xml --help

In [ ]:
smtag-convert2th --help

In [ ]:
smtag-meta --help

In [ ]:
smtag-eval --help

In [ ]:
smtag-predict --help

## Preparation of corpus of xml documents and images

In [ ]:
smtag-neo2xml -l10000 -f 181203all # on large sd-graph with 30000 panels from 1100 papers

In [ ]:
smtag-ocr # run only once

## Small molecules

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 -y ".//sd-tag[@type='molecule']" \
-f 3X_L600_molecule

In [ ]:
smtag-meta -f 3X_L600_molecule \
-E120 -Z128 -R0.01 \
-o small_molecule

saved as `3X_L600_molecule_small_molecule_2018-12-05-12-49.sddl` (size: 100167)

In [8]:
cp models/3X_L600_molecule_small_molecule_2018-12-05-12-49.zip rack/small_molecule.zip

## Gene products

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 -y ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-f 3X_L600_geneprod

In [ ]:
smtag-meta -f 3X_L600_geneprod \
-E120 -Z128 -R0.01 \
-o geneprod

model saved under `3X_L600_geneprod_geneprod_2018-12-05-00-32.sddl`

In [9]:
cp models/3X_L600_geneprod_geneprod_2018-12-05-00-32.zip rack/geneprod.zip

In [34]:
smtag-eval -f 3X_L600_geneprod -m 3X_L600_geneprod_geneprod_2018-12-05-00-32.zip -S -T

model: 3X_L600_geneprod_geneprod_2018-12-05-00-32.zip, testset: 3X_L600_geneprod, tokenization: False
opt from model namebase=3X_L600_geneprod; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['geneprod']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=1
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [21]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Datas

In [35]:
smtag-eval -f 3X_L600_geneprod -m 3X_L600_geneprod_geneprod_2018-12-05-00-32.zip

model: 3X_L600_geneprod_geneprod_2018-12-05-00-32.zip, testset: 3X_L600_geneprod, tokenization: True
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [21]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
12249 text examples of size 640
22 output features (out-channels).
Creating dataset with selected features entity: 'geneprod' (geneprod), and shuffling 12249 examples.
Generating dataset with 12249 examples
[████████████████████████████████████████████████████████████] 100% ...loading example 12222 in position 12248)
done

[█████████

## Subcellular

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 -y ".//sd-tag[@type='subcellular']" \
-f 3X_L600_subcellular

In [ ]:
smtag-meta -f 3X_L600_subcellular \
-E120 -Z128 -R0.01 \
-o subcellular

save as `3X_L600_subcellular_subcellular_2018-12-05-18-25.sddl` (size: 100167)`

In [10]:
cp models/3X_L600_subcellular_subcellular_2018-12-05-18-25.zip rack/subcellular.zip

In [37]:
smtag-eval -f 3X_L600_subcellular -m 3X_L600_subcellular_subcellular_2018-12-05-18-25.zip -S -T

model: 3X_L600_subcellular_subcellular_2018-12-05-18-25.zip, testset: 3X_L600_subcellular, tokenization: False
opt from model namebase=3X_L600_subcellular; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['subcellular']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=1
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [3]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the 

In [38]:
smtag-eval -f 3X_L600_subcellular -m 3X_L600_subcellular_subcellular_2018-12-05-18-25.zip

model: 3X_L600_subcellular_subcellular_2018-12-05-18-25.zip, testset: 3X_L600_subcellular, tokenization: True
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [3]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
3303 text examples of size 640
22 output features (out-channels).
Creating dataset with selected features entity: 'subcellular' (subcellular), and shuffling 3303 examples.
Generating dataset with 3303 examples
[████████████████████████████████████████████████████████████] 100% ...loading example 2588 in position 3302)
done

[

## Cell

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 -y ".//sd-tag[@type='cell']" \
-f 3X_L600_cell

In [ ]:
smtag-meta -f 3X_L600_cell \
-E120 -Z128 -R0.01 \
-o cell

saved `3X_L600_cell_cell_2018-12-06-18-05.sddl` (size: 100167)

In [11]:
cp models/3X_L600_cell_cell_2018-12-06-18-05.zip rack/cell.zip

In [39]:
smtag-eval -f 3X_L600_cell -m 3X_L600_cell_cell_2018-12-06-18-05.zip -S -T

model: 3X_L600_cell_cell_2018-12-06-18-05.zip, testset: 3X_L600_cell, tokenization: False
opt from model namebase=3X_L600_cell; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['cell']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=1
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [4]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
8499 t

In [40]:
smtag-eval -f 3X_L600_cell -m 3X_L600_cell_cell_2018-12-06-18-05.zip

model: 3X_L600_cell_cell_2018-12-06-18-05.zip, testset: 3X_L600_cell, tokenization: True
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [4]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
6720 text examples of size 640
22 output features (out-channels).
Creating dataset with selected features entity: 'cell' (cell), and shuffling 6720 examples.
Generating dataset with 6720 examples
[████████████████████████████████████████████████████████████] 100% ...loading example 819 in position 6719))
done

[███████████████████████████████████

## Tissue 

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 -y ".//sd-tag[@type='tissue']" \
-f 3X_L600_tissue

In [ ]:
smtag-meta -f 3X_L600_tissue \
-E120 -Z128 -R0.01 \
-o tissue

saved `3X_L600_tissue_tissue_2018-12-06-23-58.sddl` (size: 100167)

In [12]:
cp models/3X_L600_tissue_tissue_2018-12-06-23-58.zip rack/tissue.zip

In [41]:
smtag-eval -f 3X_L600_tissue -m 3X_L600_tissue_tissue_2018-12-06-23-58.zip -S -T

model: 3X_L600_tissue_tissue_2018-12-06-23-58.zip, testset: 3X_L600_tissue, tokenization: False
opt from model namebase=3X_L600_tissue; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['tissue']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=1
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [5]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensio

In [42]:
smtag-eval -f 3X_L600_tissue -m 3X_L600_tissue_tissue_2018-12-06-23-58.zip

model: 3X_L600_tissue_tissue_2018-12-06-23-58.zip, testset: 3X_L600_tissue, tokenization: True
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [5]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
2862 text examples of size 640
22 output features (out-channels).
Creating dataset with selected features entity: 'tissue' (tissue), and shuffling 2862 examples.
Generating dataset with 2862 examples
[████████████████████████████████████████████████████████████] 100% ...loading example 420 in position 2861))
done

[█████████████████████████

## Organism 

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 -y ".//sd-tag[@type='organism']" \
-f 3X_L600_organism

In [ ]:
smtag-meta -f 3X_L600_organism \
-E120 -Z128 -R0.01 \
-o organism

saved `3X_L600_organism_organism_2018-12-07-04-36.sddl` (size: 100167)

In [15]:
cp models/3X_L600_organism_organism_2018-12-07-04-36.zip rack/organism.zip

In [44]:
smtag-eval -f 3X_L600_organism -m 3X_L600_organism_organism_2018-12-07-04-36.zip -S -T

model: 3X_L600_organism_organism_2018-12-07-04-36.zip, testset: 3X_L600_organism, tokenization: False
opt from model namebase=3X_L600_organism; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['organism']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=1
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [6]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Datase

In [45]:
smtag-eval -f 3X_L600_organism -m 3X_L600_organism_organism_2018-12-07-04-36.zip

model: 3X_L600_organism_organism_2018-12-07-04-36.zip, testset: 3X_L600_organism, tokenization: True
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [6]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
4059 text examples of size 640
22 output features (out-channels).
Creating dataset with selected features entity: 'organism' (organism), and shuffling 4059 examples.
Generating dataset with 4059 examples
[████████████████████████████████████████████████████████████] 100% ...loading example 1341 in position 4058)
done

[███████████████

## Diseases 

In [ ]:
smtag-convert2th -L1200 -X5 -b -c NCBI_disease -f 5X_L1200_NCBI_disease

In [ ]:
smtag-meta -E120 -Z128 -R0.01 -o disease -f 5X_L1200_NCBI_disease

saved `5X_L1200_NCBI_disease_disease_2018-12-12-07-44.sddl` (size: 100167)

In [14]:
cp models/5X_L1200_NCBI_disease_disease_2018-12-12-07-44.zip rack/disease.zip

In [47]:
smtag-eval -f 5X_L1200_NCBI_disease -m 5X_L1200_NCBI_disease_disease_2018-12-12-07-44.zip -S -T

model: 5X_L1200_NCBI_disease_disease_2018-12-12-07-44.zip, testset: 5X_L1200_NCBI_disease, tokenization: False
opt from model namebase=5X_L1200_NCBI_disease; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['disease']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=1
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [18]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the d

In [48]:
smtag-eval -f 5X_L1200_NCBI_disease -m 5X_L1200_NCBI_disease_disease_2018-12-12-07-44.zip

model: 5X_L1200_NCBI_disease_disease_2018-12-12-07-44.zip, testset: 5X_L1200_NCBI_disease, tokenization: True
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [18]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
1190 text examples of size 1240
22 output features (out-channels).
Creating dataset with selected features disease: 'disease' (), and shuffling 1190 examples.
Generating dataset with 1190 examples
[████████████████████████████████████████████████████████████] 100% ...loading example 533 in position 1189)
done

[█████████████

## Experimental assay 

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 -y ".//sd-tag[@category='assay']" \
-f 3X_L600_assay

In [ ]:
smtag-meta -f 3X_L600_assay \
-E120 -Z128 -R0.01 \
-o assay

saved `3X_L600_assay_assay_2018-12-06-06-59.sddl` (size: 100167)

In [19]:
cp models/3X_L600_assay_assay_2018-12-06-06-59.zip rack/exp_assay.zip

In [50]:
smtag-eval -f 3X_L600_assay -m 3X_L600_assay_assay_2018-12-06-06-59.zip -S -T

model: 3X_L600_assay_assay_2018-12-06-06-59.zip, testset: 3X_L600_assay, tokenization: False
opt from model namebase=3X_L600_assay; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['assay']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=1
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [14]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:


In [52]:
smtag-eval -f 3X_L600_assay -m 3X_L600_assay_assay_2018-12-06-06-59.zip

model: 3X_L600_assay_assay_2018-12-06-06-59.zip, testset: 3X_L600_assay, tokenization: True
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [14]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
9510 text examples of size 640
22 output features (out-channels).
Creating dataset with selected features assay: 'assay' (), and shuffling 9510 examples.
Generating dataset with 9510 examples
[████████████████████████████████████████████████████████████] 100% ...loading example 3663 in position 9509)
done

[███████████████████████████████████

## Intervention-assay geneprod 

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 \
-y ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-e ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-A ".//sd-tag[@role='intervention']",\
".//sd-tag[@role='assayed']",\
".//sd-tag[@role='normalizing']",\
".//sd-tag[@role='experiment']",\
".//sd-tag[@role='component']", \
-f 3X_L600_geneprod_anonym_not_reporter

In [ ]:
smtag-meta -f 3X_L600_geneprod_anonym_not_reporter \
-E120 -Z128 -R0.01 \
-o intervention,assayed

saved `3X_L600_geneprod_anonym_not_reporter_intervention_assayed_2018-12-10-00-33.sddl` (size: 100481)

In [20]:
cp models/3X_L600_geneprod_anonym_not_reporter_intervention_assayed_2018-12-10-00-33.zip rack/role_geneprod.zip

In [54]:
smtag-eval -f 3X_L600_geneprod_anonym_not_reporter -m 3X_L600_geneprod_anonym_not_reporter_intervention_assayed_2018-12-10-00-33.zip -S -T

model: 3X_L600_geneprod_anonym_not_reporter_intervention_assayed_2018-12-10-00-33.zip, testset: 3X_L600_geneprod_anonym_not_reporter, tokenization: False
opt from model namebase=3X_L600_geneprod_anonym_not_reporter; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['intervention', 'assayed']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=2
nf.output= 2
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [8, 9]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the 

In [55]:
smtag-eval -f 3X_L600_geneprod_anonym_not_reporter -m 3X_L600_geneprod_anonym_not_reporter_intervention_assayed_2018-12-10-00-33.zip

model: 3X_L600_geneprod_anonym_not_reporter_intervention_assayed_2018-12-10-00-33.zip, testset: 3X_L600_geneprod_anonym_not_reporter, tokenization: True
nf.output= 2
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [8, 9]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
12249 text examples of size 640
22 output features (out-channels).
Creating dataset with selected features entity: 'intervention' (intervention), entity: 'assayed' (assayed), and shuffling 12249 examples.
Generating dataset with 12249 examples
[████████████████████████████████████

## Geneprod Reporter

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 \
-y ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-e ".//sd-tag[@type='gene']",".//sd-tag[@type='protein']" \
-f 3X_L600_geneprod_reporter

In [2]:
smtag-meta -f 3X_L600_geneprod_reporter \
-E120 -Z128 -R0.01 \
-o reporter

saved `3X_L600_geneprod_reporter_reporter_2018-12-08-04-24.sddl` (size: 100279)

In [21]:
cp models/3X_L600_geneprod_reporter_reporter_2018-12-08-04-24.zip rack/reporter_geneprod.zip

In [57]:
smtag-eval -f 3X_L600_geneprod_reporter -m 3X_L600_geneprod_reporter_reporter_2018-12-08-04-24.zip -S -T

model: 3X_L600_geneprod_reporter_reporter_2018-12-08-04-24.zip, testset: 3X_L600_geneprod_reporter, tokenization: False
opt from model namebase=3X_L600_geneprod_reporter; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['reporter']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=1
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [11]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the exa

In [58]:
smtag-eval -f 3X_L600_geneprod_reporter -m 3X_L600_geneprod_reporter_reporter_2018-12-08-04-24.zip

model: 3X_L600_geneprod_reporter_reporter_2018-12-08-04-24.zip, testset: 3X_L600_geneprod_reporter, tokenization: True
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [11]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
12249 text examples of size 640
22 output features (out-channels).
Creating dataset with selected features entity: 'reporter' (reporter), and shuffling 12249 examples.
Generating dataset with 12249 examples
[████████████████████████████████████████████████████████████] 100% ...loading example 9551 in position 12248)

## Intervention-assay small molecule

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 \
-y ".//sd-tag[@type='molecule']" \
-e ".//sd-tag[@type='molecule']" \
-A ".//sd-tag[@type='molecule']" \
-f 3X_L600_molecule_anonym

In [ ]:
smtag-meta -f 3X_L600_molecule_anonym \
-E120 -Z128 -R0.01 \
-o intervention,assayed

saved `3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.sddl` (size: 100315)

In [22]:
cp models/3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.zip rack/role_small_molecule.zip

In [59]:
smtag-eval -f 3X_L600_molecule_anonym -m 3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.zip -S -T

model: 3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.zip, testset: 3X_L600_molecule_anonym, tokenization: False
opt from model namebase=3X_L600_molecule_anonym; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['intervention', 'assayed']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=2
nf.output= 2
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [8, 9]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as prov

In [60]:
smtag-eval -f 3X_L600_molecule_anonym -m 3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.zip

model: 3X_L600_molecule_anonym_intervention_assayed_2018-12-13-05-20.zip, testset: 3X_L600_molecule_anonym, tokenization: True
nf.output= 2
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [8, 9]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
5118 text examples of size 640
22 output features (out-channels).
Creating dataset with selected features entity: 'intervention' (intervention), entity: 'assayed' (assayed), and shuffling 5118 examples.
Generating dataset with 5118 examples
[████████████████████████████████████████████████████████████] 100

## Panel start

In [ ]:
smtag-convert2th -c 181203all \
--noocr -E ".//figure-caption" \
-L1200 -X5 -f 5X_L1200_figure_level_no_ocr

In [ ]:
smtag-meta -f 5X_L1200_figure_level_no_ocr \
-E120 -Z128 -R0.01 \
-o panel_start

saved `5X_L1200_figure_level_no_ocr_panel_start_2018-12-12-00-25.sddl` (size: 100167)

In [23]:
cp models/5X_L1200_figure_level_no_ocr_panel_start_2018-12-12-00-25.zip rack/panel_start.zip

In [62]:
smtag-eval -f 5X_L1200_figure_level_no_ocr -m 5X_L1200_figure_level_no_ocr_panel_start_2018-12-12-00-25.zip -S -T

model: 5X_L1200_figure_level_no_ocr_panel_start_2018-12-12-00-25.zip, testset: 5X_L1200_figure_level_no_ocr, tokenization: False
opt from model namebase=5X_L1200_figure_level_no_ocr; modelname=; learning_rate=0.01; dropout=0.1; epochs=120; minibatch_size=128; selected_features=['panel_start']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; use_ocr_context=; use_viz_context=False; nf_input=32; nf_output=1
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [19]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance i

In [63]:
smtag-eval -f 5X_L1200_figure_level_no_ocr -m 5X_L1200_figure_level_no_ocr_panel_start_2018-12-12-00-25.zip

model: 5X_L1200_figure_level_no_ocr_panel_start_2018-12-12-00-25.zip, testset: 5X_L1200_figure_level_no_ocr, tokenization: True
nf.output= 1
nf.input= 32
index_of_collapsed_feature= None
index_of_overlap_feature None
concept2index self.selected_features [19]
concept2index self.collapsed_features []
concept2index self.overlap_features []
Loading features.pyth as features for the dataset.
Loading textcoded.pyth as encoded text for the dataset.
Loading ocrcontext.pyth as image-based OCR data.
No image-based visual context data available.
Loading text.txt for the original texts of the dataset.
Loading provenance.txt as provenance info for the examples in the dataset.
Dataset dimensions:
6255 text examples of size 1240
22 output features (out-channels).
Creating dataset with selected features : 'panel_start' (), and shuffling 6255 examples.
Generating dataset with 6255 examples
[████████████████████████████████████████████████████████████] 100% ...loading example 3973 in position 6254)
done

## Special models (experimental)

### Pan entity measured variable

In [ ]:
smtag-convert2th -c 181203all \
-L600 -X3 \
-A ".//sd-tag[@role]" \
-f 3X_L600_all_entities_role_anonym

In [ ]:
smtag-meta -f 3X_L600_all_entities_role_anonym \
-E120 -Z128 -R0.01 \
-o intervention,assayed

### Multi-figure level intervention-assayed model

In [ ]:
smtag-convert2th -c 181203all \
--noocr -E "." \
-L5000 -X1 -f 1X_L5000_doc_level_no_ocr

## Rack for SmartTag engine

In [61]:
ls rack

cell.zip		organism.zip		role_small_molecule.zip
disease.zip		panel_start.zip		small_molecule.zip
exp_assay.zip		reporter_geneprod.zip	subcellular.zip
geneprod.zip		role_geneprod.zip	tissue.zip


## Hyperparam scans

In [ ]:
smtag-meta -E50 -o panel_start -H depth,kernel -I 25 -f 5X_L1200_entities -w /efs/smtag # after data transfer to GPU machine

Winning combo:
    4,4,4,4
    
But a lot of variability. Initialization issues?

In [ ]:
smtag-meta -E50 -o geneprod -H kernel,depth -I 25 -f 5X_L1200_gene_protein -w /efs/smtag # after transfer to GPU machine

Winning combo with `f1=0.80` after 50 epochs, no overfitting with valid:

    namebase=5X_L1200_gene_protein; modelname=; learning_rate=0.01000000000000001; epochs=50; minibatch_size=32; selected_features=['geneprod']; collapsed_features=[]; overlap_features=[]; features_as_input=[]; nf_table=[8, 8, 8]; pool_table=[2, 2, 2]; kernel_table=[6, 6, 6]; dropout=0.1; validation_fraction=0.2; nf_input=32; nf_output=1